In [31]:
# Classify PTT (Tech_job broad) posts to content type by RNN model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
board = 'Tech_job'
now_time = '1519748408.0598423'
board_time = board + '_' + now_time

board_dir = os.path.join(base_dir, board_time)
if not os.path.exists(board_dir):
    os.makedirs(board_dir)
content_dir = os.path.join(board_dir, 'content')
if not os.path.exists(content_dir):
    os.makedirs(content_dir)
push_dir = os.path.join(board_dir, 'push')
if not os.path.exists(push_dir):
    os.makedirs(push_dir)

type_label = ['討論', '心得', '請益', '面試', '聘書', '徵才', '新聞', '情報']
type_newlabel = ['討論', '心得', '請益', '情報']
label_num = len(type_label)
label_newnum = len(type_newlabel)

def type2number(type_word):
    type_num = 0
    for type_name in type_newlabel:
        if type_word.find(type_name)>=0:
            break
        type_num = type_num + 1
    return type_num

def symbol_filter(crazystring):
    filter_words=' !"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'　，。！：；、？﹝﹞「」『』（）｛｝［］【】《》“”‘’＼｜〝〞‵′＋－＊／＝≦≧＿＠＃＄％︿＆～§◎．※ㄧ↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔'
    is_filtered = True
    for crazychar in crazystring:
        if filter_words.find(crazychar)<0:
            is_filtered = False
    return is_filtered

In [47]:
#整理、修改原始數據
dfs = pd.read_csv(os.path.join(board_dir, now_time + '.csv'))
dfs_len = len(dfs)

print('Min: ', dfs.iloc[:, 5].min())
print('0.1: ', dfs.iloc[:, 5].quantile(0.1))
print('0.25: ', dfs.iloc[:, 5].quantile(0.25))
print('Mean: ', dfs.iloc[:, 5].mean())
print('0.75: ', dfs.iloc[:, 5].quantile(0.75))
print('0.9: ', dfs.iloc[:, 5].quantile(0.9))
print('0.95: ', dfs.iloc[:, 5].quantile(0.95))
print('Max: ', dfs.iloc[:, 5].max())
print('Std: ', dfs.iloc[:, 5].std())

max_length = 512
text_type = []
text_content = []
type_name_list = [0] * label_num
type_newname_list = [0] * label_newnum
for dfs_index in range(0, dfs_len):
    read_index = dfs.ix[dfs_index, 0]
    type_name = dfs.ix[dfs_index, 1]
    title_name = dfs.ix[dfs_index, 9]
    #print(read_index, ': ', dfs.ix[dfs_index, 7], title_name, end='\r')
    for type_num in range(label_num):
        if type_name.find(type_label[type_num])>=0:
            type_name_list[type_num] = type_name_list[type_num] + 1
    
    #排除標題與副標題
    if type_name.find('問卷')>=0 or type_name.find('市調')>=0 or type_name.find('公告')>=0 or type_name.find('政見')>=0 or type_name.find('尋人')>=0 or type_name.find('協尋')>=0 or type_name.find('學術')>=0:
        print('', end='\r')
    elif type(title_name) == type('a') and (title_name.find('問卷')>=0 or type_name.find('市調')>=0 or title_name.find('公告')>=0 or title_name.find('政見')>=0 or title_name.find('尋人')>=0 or title_name.find('協尋')>=0):
        print('', end='\r')
    #整理副標題
    elif type_name.find('聘書')>=0 or type_name.find('offer')>=0 or type_name.find('Offer')>=0 or type_name.find('OFFER')>=0:
        dfs.ix[dfs_index, 1] = '心得' #聘書'
    elif type_name.find('徵才')>=0 or type_name.find('徵求')>=0 or type_name.find('求才')>=0 or type_name.find('招募')>=0 or type_name.find('誠徵')>=0 or type_name.find('急徵')>=0 or type_name.find('代徵')>=0 or type_name.find('職缺')>=0:
        dfs.ix[dfs_index, 1] = '情報' #'徵才'
    elif type_name.find('台北')>=0 or type_name.find('新北')>=0 or type_name.find('雙北')>=0 or type_name.find('北部')>=0 or type_name.find('中部')>=0 or type_name.find('南部')>=0 or type_name.find('新竹')>=0 or type_name.find('北美')>=0 or type_name.find('新加坡')>=0:
        dfs.ix[dfs_index, 1] = '情報' #'徵才'
    elif type_name.find('心得')>=0 or type_name.find('心情')>=0 or type_name.find('感想')>=0 or type_name.find('有感')>=0 or type_name.find('血淚史')>=0:
        dfs.ix[dfs_index, 1] = '心得'
    elif type_name.find('面試')>=0 or type_name.find('面談')>=0:
        dfs.ix[dfs_index, 1] = '心得' #'面試'
    elif type_name.find('新聞')>=0 or type_name.find('專欄')>=0 or type_name.find('辛聞')>=0 or type_name.find('社論')>=0:
        dfs.ix[dfs_index, 1] = '情報' #'新聞'
    elif type_name.find('請益')>=0 or type_name.find('解籤')>=0 or type_name.find('問題')>=0 or type_name.find('問卦')>=0 or type_name.find('請教')>=0 or type_name.find('詢問')>=0 or type_name.find('求助')>=0 or type_name.find('疑問')>=0 or type_name.find('提問')>=0 or type_name.find('請問')>=0 or type_name.find('請求')>=0 or type_name.find('協助')>=0 or type_name.find('問題')>=0:
        dfs.ix[dfs_index, 1] = '請益'
    elif type_name.find('情報')>=0 or type_name.find('開獎')>=0 or type_name.find('資訊')>=0 or type_name.find('分享')>=0 or type_name.find('活動')>=0 or type_name.find('講座')>=0 or type_name.find('職訓')>=0 or type_name.find('課程')>=0 or type_name.find('座談會')>=0:
        dfs.ix[dfs_index, 1] = '情報'
    elif type_name.find('討論')>=0 or type_name.find('見鬼')>=0 or type_name.find('夢境')>=0 or type_name.find('爆卦')>=0  or type_name.find('爆掛')>=0 or type_name.find('閒聊')>=0 or type_name.find('聊天')>=0:
        dfs.ix[dfs_index, 1] = '討論'
    #整理標題
    elif type(title_name) != type('a'):
        continue
    elif title_name.find('聘書')>=0 or title_name.find('offer')>=0 or title_name.find('Offer')>=0 or title_name.find('OFFER')>=0:
        dfs.ix[dfs_index, 1] = '心得' #'聘書'
    elif title_name.find('徵才')>=0 or title_name.find('徵求')>=0 or title_name.find('求才')>=0 or title_name.find('招募')>=0  or title_name.find('誠徵')>=0 or title_name.find('急徵')>=0 or title_name.find('代徵')>=0 or title_name.find('職缺')>=0 or title_name.find('高薪徵')>=0:
        dfs.ix[dfs_index, 1] = '情報' #'徵才'
    elif title_name.find('心得')>=0 or title_name.find('心情')>=0 or title_name.find('感想')>=0 or title_name.find('有感')>=0 or title_name.find('血淚史')>=0:
        dfs.ix[dfs_index, 1] = '心得'
    elif title_name.find('面試')>=0 or title_name.find('面談')>=0:
        dfs.ix[dfs_index, 1] = '心得' #'面試'
    elif title_name.find('新聞')>=0 or title_name.find('專欄')>=0 or title_name.find('辛聞')>=0 or title_name.find('社論')>=0:
        dfs.ix[dfs_index, 1] = '情報' #'新聞'
    elif title_name.find('請益')>=0 or title_name.find('解籤')>=0 or title_name.find('問題')>=0 or title_name.find('問卦')>=0 or title_name.find('請教')>=0 or title_name.find('詢問')>=0 or title_name.find('求助')>=0 or title_name.find('疑問')>=0 or title_name.find('提問')>=0 or title_name.find('請問')>=0 or title_name.find('請求')>=0 or title_name.find('協助')>=0 or title_name.find('問題')>=0:
        dfs.ix[dfs_index, 1] = '請益'
    elif title_name.find('情報')>=0 or title_name.find('開獎')>=0 or title_name.find('分享')>=0 or title_name.find('活動')>=0 or title_name.find('講座')>=0 or title_name.find('職訓')>=0 or title_name.find('課程')>=0 or title_name.find('座談會')>=0:
        dfs.ix[dfs_index, 1] = '情報'
    elif title_name.find('討論')>=0 or title_name.find('見鬼')>=0 or title_name.find('夢境')>=0 or title_name.find('爆卦')>=0 or title_name.find('爆掛')>=0 or title_name.find('閒聊')>=0 or title_name.find('聊天')>=0:
        dfs.ix[dfs_index, 1] = '討論'
    #最後分類
    elif type_name.find('無題')>=0 or type_name.find('其他')>=0:
        dfs.ix[dfs_index, 1] = '討論'
    else:
        print(type_name, ': ', dfs.ix[dfs_index, 7], title_name)
    
    type_newname = dfs.ix[dfs_index, 1]
    for type_num in range(label_newnum):
        if type_newname.find(type_newlabel[type_num])>=0:
            type_newname_list[type_num] = type_newname_list[type_num] + 1

print(type_label,', 其他, 公告 (original)')
print(type_name_list)
print(type_newlabel,', 其他, 公告')
print(type_newname_list)

dfs.to_csv(os.path.join(board_dir, now_time + '_fix.csv'), encoding = 'utf-8-sig', index=False)

Min:  1
0.1:  91.0
0.25:  146.0
Mean:  550.8913021345912
0.75:  698.0
0.9:  1181.0
0.95:  1674.0
Max:  19644
Std:  728.991418870587
轉錄 :  https://www.ptt.cc/bbs/Tech_Job/M.1084713565.A.D36.html 職位中英文
轉錄 :  https://www.ptt.cc/bbs/Tech_Job/M.1143282919.A.F48.html [就可] 嫁給工程師的好處多?! ^_^
無關 :  https://www.ptt.cc/bbs/Tech_Job/M.1220011862.A.651.html boylover兄臺
檢舉 :  https://www.ptt.cc/bbs/Tech_Job/M.1249658939.A.5D9.html  鬧板 
申請 :  https://www.ptt.cc/bbs/Tech_Job/M.1255582856.A.ADA.html  Tech_Job板申請新增板主
論卦 :  https://www.ptt.cc/bbs/Tech_Job/M.1259327542.A.ACA.html  通膨經濟學、兩萬二、與台灣人大滅絕
建議 :  https://www.ptt.cc/bbs/Tech_Job/M.1259482668.A.A6E.html 精華區"公司"部分
轉文 :  https://www.ptt.cc/bbs/Tech_Job/M.1270320846.A.2CF.html

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


 派遣工作 缺點(轉)  
轉潑 :  https://www.ptt.cc/bbs/Tech_Job/M.1270482835.A.038.html  職場人都該好好的看這一篇文章
轉潑 :  https://www.ptt.cc/bbs/Tech_Job/M.1270483282.A.862.html 職場人都該好好的看這一篇文章[2]
好文 :  https://www.ptt.cc/bbs/Tech_Job/M.1270490410.A.01A.html  請把我們當人看-ㄧ個洋華員工的心聲
申請 :  https://www.ptt.cc/bbs/Tech_Job/M.1314891379.A.5B9.html  Tech_Job 板發文限制申請
申請 :  https://www.ptt.cc/bbs/Tech_Job/M.1318867663.A.EDC.html  Tech_Job板申請新增板主
申請 :  https://www.ptt.cc/bbs/Tech_Job/M.1321711030.A.59D.html  Tech_Job新增板主mmkntust
新增 :  https://www.ptt.cc/bbs/Tech_Job/M.1325561824.A.F07.html  Tech_Job 新增板主 mmkntust/harrygp/lovewsc
猶豫 :  https://www.ptt.cc/bbs/Tech_Job/M.1340671920.A.E57.html  工程師？還是國考？
非關科技版 :  https://www.ptt.cc/bbs/Tech_Job/M.1342092631.A.181.html  風力+太陽能 並聯
週刊 :  https://www.ptt.cc/bbs/Tech_Job/M.1342684240.A.12B.html  台灣公務員與死老百姓的收入財富比較
轉錄 :  https://www.ptt.cc/bbs/Tech_Job/M.1345795739.A.A35.html 外商來台設立研發中心, 還要政府補助? ꨠ
轉錄 :  https://www.ptt.cc/bbs/Tech_Job/M.1345820997.A.AD9.html 外商來台設立研發中心, 還要政府補助?  
抱怨 :

免費 :  https://www.ptt.cc/bbs/Tech_Job/M.1488777506.A.245.html 工作人生顧問
ＦＢ :  https://www.ptt.cc/bbs/Tech_Job/M.1488857447.A.24A.html  中華民國行政院FB:中台灣機械之都啟航
舊聞 :  https://www.ptt.cc/bbs/Tech_Job/M.1489677705.A.676.html  下班傳LINE算加班 資方痛批：「刁勞」越
公司 :  https://www.ptt.cc/bbs/Tech_Job/M.1490262082.A.661.html  大船集團-達航科技
公司 :  https://www.ptt.cc/bbs/Tech_Job/M.1490357939.A.F46.html  大船集團-達航科技
公司 :  https://www.ptt.cc/bbs/Tech_Job/M.1492788084.A.060.html  群燿科技
公司 :  https://www.ptt.cc/bbs/Tech_Job/M.1494545379.A.110.html  豐軒科技股份有限公司
ＦＢ :  https://www.ptt.cc/bbs/Tech_Job/M.1496071077.A.1BE.html  時力:數位金融要發展 就從告別財金幫開始
ＦＢ :  https://www.ptt.cc/bbs/Tech_Job/M.1496453160.A.228.html  余宛如-開放外國白領來台才能接軌國際
轉錄 :  https://www.ptt.cc/bbs/Tech_Job/M.1496712119.A.242.html  前瞻計畫為啥沒遊戲產業?
問券 :  https://www.ptt.cc/bbs/Tech_Job/M.1498810356.A.F7D.html 救救研究生 順便抽餐券
轉錄 :  https://www.ptt.cc/bbs/Tech_Job/M.1499303211.A.F8C.html  機能衣大躍進，紡織矽谷的崛起
公司 :  https://www.ptt.cc/bbs/Tech_Job/M.1500953352.A.EDA.html  圓凱科技實業
公司 :  https

In [52]:
#Select 'type' of posts to save.
dfs = pd.read_csv(os.path.join(board_dir, now_time + '_fix.csv'))
word_num_min = 100
word_num_max = 2048
text_content = []
for dfs_index in range(0, len(dfs)):
    #Remove the post if it's word number is too large or small.
    word_num = int(dfs.ix[dfs_index, 5])
    push_num = int(dfs.ix[dfs_index, 6])
    if word_num<word_num_min or word_num>word_num_max:
        dfs.drop([dfs_index], inplace=True)
        continue
    #Remove the post if the type is not needed.
    is_label = False
    this_label = dfs.ix[dfs_index, 1]
    for find_label in type_newlabel:
        if this_label.find(find_label)>=0:
            is_label = True
            break
    #Remove the post if it's no content.
    if is_label:
        try:
            file = open(os.path.join(content_dir, str(dfs_index) + '.csv'), 'r', encoding = 'utf-8-sig')
            csvCursor = csv.reader(file)
        except BaseException:
            is_label = False
    if is_label:
        text = ''
        for row in csvCursor:
            try:
                text = text + row[0]
            except BaseException:
                continue
        text_content.append(text[0:word_num_max])
        file.close()
    else: #Remove the row if it's no need.
        dfs.drop([dfs_index], inplace=True)

dfs.to_csv(os.path.join(board_dir, now_time + '_pandas.csv'), encoding = 'utf-8-sig', index=False)

dfs_len = len(dfs)
print(dfs_len)
print(type_newlabel)
type_newname_list = [0] * label_newnum
for dfs_index in range(0, dfs_len):
    for type_num in range(0, label_newnum):
        if type_newlabel[type_num] == dfs.iloc[dfs_index, 1]:
            type_newname_list[type_num] = type_newname_list[type_num] + 1

print(type_newname_list)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


47991
['討論', '心得', '請益', '情報']
[7071, 4816, 25358, 10746]


In [76]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
import jieba

jieba.set_dictionary('C:/Users/User/Anaconda3/Lib/site-packages/jieba/dict.txt.big')
dfs_pandas = pd.read_csv(os.path.join(board_dir, now_time + '_pandas.csv'))
dfs_len = len(dfs_pandas)

print('Min: ', dfs_pandas.iloc[:, 5].min())
print('0.1: ', dfs_pandas.iloc[:, 5].quantile(0.1))
print('0.25: ', dfs_pandas.iloc[:, 5].quantile(0.25))
print('Mean: ', dfs_pandas.iloc[:, 5].mean())
print('0.75: ', dfs_pandas.iloc[:, 5].quantile(0.75))
print('0.9: ', dfs_pandas.iloc[:, 5].quantile(0.9))
print('0.95: ', dfs_pandas.iloc[:, 5].quantile(0.95))
print('Max: ', dfs_pandas.iloc[:, 5].max())
print('Std: ', dfs_pandas.iloc[:, 5].std())

max_length = 1024
text_type = []
text_content = []
dict_list = []
type_words = [0] * label_newnum
dict_entropy = {}
for dict_ind in range(label_newnum):
    dict_list.append({})
    
for dfs_index in range(0, dfs_len):
    read_index = dfs_pandas.ix[dfs_index, 0]
    read_type = type2number(dfs_pandas.ix[dfs_index, 1])
    file = open(os.path.join(content_dir, str(read_index) + '.csv'), 'r', encoding = 'utf-8-sig')
    csvCursor = csv.reader(file)
    text = ''
    for row in csvCursor:
        try:
            #Remove http.
            while row[0].find('http')>=0:
                http_str=row[0].find('http')
                http_end=row[0].find('\n', http_str)
                row[0] = row[0].replace(row[0][http_str:http_end], '')
            text = text + row[0]
        except BaseException:
                continue
    text.encode('utf-8-sig')
    text_jieba = jieba.cut(text, cut_all=False)
    for word in text_jieba:
        if symbol_filter(word):
            continue
        type_words[read_type] = type_words[read_type] + 1
        if word not in dict_entropy.keys():
            dict_entropy[word] = 0.0
            for label_ind in range(label_newnum):
                dict_list[label_ind][word] = int(label_ind == read_type)
        else:
            dict_list[read_type][word] = dict_list[read_type][word] + 1
    text_content.append(text_jieba)
    text_type.append(read_type)
    file.close()

print('Vocab number: ',len(dict_entropy))
energy_limit = 6000
dict_energy = {}
type_rate = [0.0] * label_newnum
value_sum = sum(type_words)
for label_ind in range(label_newnum):
    type_rate[label_ind] = type_words[label_ind] / value_sum

for dict_word in dict_entropy:
    value_sum = 0
    for label_ind in range(label_newnum):
        value_sum = value_sum + dict_list[label_ind][dict_word]
    dict_energy[dict_word] = value_sum
    entropy = 0.0
    for label_ind in range(label_newnum):
        prop = type_rate[label_ind] * dict_list[label_ind][dict_word] / value_sum
        entropy = entropy - prop * np.log(prop)
    dict_entropy[dict_word] = entropy

list_energy = sorted(dict_energy.items(), key=lambda dict_energy: dict_energy[1], reverse=True)
energy_limit_num = list_energy[energy_limit][1]
print('Energy limit number: ',energy_limit_num)

for vocab_index in dict_energy:
    if dict_energy[vocab_index] < energy_limit_num:
        del dict_entropy[vocab_index]
list_entropy = sorted(dict_entropy.items(), key=lambda dict_entropy: dict_entropy[1], reverse=True)

for vocab_index in range(len(dict_entropy)):
    if vocab_index % 1000 == 0 or vocab_index < 20:
        print(vocab_index, list_entropy[vocab_index], dict_energy[list_entropy[vocab_index][0]])
    vocab_index = vocab_index + 1

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
Building prefix dict from C:\Users\User\Anaconda3\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ufa6ae29b0cbce8b45e006c7fa30eaaf8.cache


Min:  100
0.1:  128.0
0.25:  188.0
Mean:  511.8679127336376
0.75:  713.0
0.9:  1095.0
0.95:  1364.0
Max:  2048
Std:  405.7673342208644


Loading model cost 1.192 seconds.
Prefix dict has been built succesfully.


Vocab number:  205227


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in log
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars


Energy limit number:  137
0 ('放在', 0.6837956161701394) 578
1 ('PTT', 0.68375532376887249) 709
2 ('脫離', 0.68370896796534719) 189
3 ('不少', 0.68365888523370166) 2354
4 ('以上', 0.68362222125712624) 7883
5 ('提早', 0.68358267698926711) 474
6 ('設定', 0.68355606106331046) 460
7 ('35', 0.68351044299716146) 1034
8 ('同樣', 0.68348796927123734) 1257
9 ('的確', 0.68346191497158904) 974
10 ('時', 0.68342097592672224) 8906
11 ('資深', 0.68339733813153913) 2081
12 ('做法', 0.68338506778377428) 410
13 ('重要', 0.68338349581380142) 4292
14 ('看似', 0.68333432415464568) 200
15 ('包', 0.68325458178998932) 712
16 ('唯一', 0.68322490693547144) 890
17 ('而言', 0.68319704511984303) 888
18 ('能力', 0.68317913789232065) 6983
19 ('強迫', 0.6831787775967868) 293
1000 ('維持', 0.59949360481600955) 1140
2000 ('產線', 0.67605255030613798) 1080
3000 ('2330', 0.44066498953645394) 320
4000 ('QA', 0.66851385353098558) 419
5000 ('layout', 0.61441875795228085) 362
6000 ('紫光', 0.43749079554499776) 451


In [56]:
d = {'OK':1,'GG':0,'JB':2}
x = d.keys()
y = d.values()
z = d.items()
print(x)
print(y)
print(z)
l = sorted(d.items(), key=lambda d: d[1], reverse=True)
print(l)
print(l[0][0])

dict_keys(['OK', 'GG', 'JB'])
dict_values([1, 0, 2])
dict_items([('OK', 1), ('GG', 0), ('JB', 2)])
[('JB', 2), ('OK', 1), ('GG', 0)]
JB


In [82]:
from sklearn.cross_validation import train_test_split
from keras.preprocessing.sequence import pad_sequences

x_train, x_test, y_train, y_test = train_test_split(text_content, text_type, test_size=0.2, random_state=9487)

y_train = np_utils.to_categorical(y_train, label_newnum)
y_test = np_utils.to_categorical(y_test, label_newnum)

# 序列模式
x_train = pad_sequences(x_train_word_ids, maxlen=max_length)
x_test = pad_sequences(x_test_word_ids, maxlen=max_length)

Vocab number:  198401
vocab_index vocab_word vocab_cnt vocab_doc
0 的 401906 45214
1 是 134519 36778
2 我 105585 24549
3 有 97402 35125
4 在 88650 31686
5 了 80607 27791
6 公司 78199 27894
7 也 71346 27138
8 都 61279 25135
9 工作 59346 23601
1000 看法 1413 1298
2000 角度 615 529
3000 ie 357 225
4000 調漲 241 162
5000 87 178 140
6000 童子 139 53
7000 潭子 111 95
8000 再有 92 88
9000 搭車 78 73
10000 二哥 67 56
11000 敢講 58 53
12000 規章 51 38
13000 air 45 32
14000 睿 41 29
15000 甄上 37 35
16000 全力支持 33 32
17000 政商 30 28
18000 傳產當 28 28
19000 損壞 26 22
20000 當奴才 24 20
21000 某私 22 22
22000 郭易 21 9
23000 有段 19 19
24000 八折 18 17
25000 趨向 17 16
26000 往日 16 16
27000 吸取 15 15
28000 壽險業 14 9
29000 通告 13 11
30000 埋首 13 8
31000 給在 12 12
32000 殺戮 11 5
33000 分心 11 11
34000 四元 10 8
35000 末位 10 7
36000 recruitment 9 9
37000 密不可分 9 9
38000 禎 9 6
39000 savage 8 8
40000 奮力 8 8
41000 必反 8 5
42000 控器 7 3
43000 半小 7 7
44000 治病 7 7
45000 傳送帶 7 2
46000 河東 6 6
47000 活過來 6 6
48000 陳俊廷 6 1
49000 人報 6 6
50000 遭法 6 2
51000 差一截 5 5
52000 關電 5 5
53

In [88]:
import os
import time
import argparse
import numpy as np
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation, Embedding, Concatenate
from keras.layers import Conv1D, Flatten, Dropout, MaxPool1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
from keras.optimizers import Adam, Adadelta
import keras.callbacks

patience = 2
num_epoch = 12
batch_size = 64
save_every = 1
pretrain = 0

model_name = 'RNN_PTT_model_fixed_' + str(time.time())
mdl_dir = os.path.join(base_dir, 'model')
if not os.path.exists(mdl_dir):
    os.makedirs(mdl_dir)
model_dir = os.path.join(mdl_dir, model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

def build_model():
    main_input = Input(shape=(max_length,), dtype='float64')
    
    embed = Embedding(8000, 300, input_length=max_length)(main_input)
    
    cnn = Conv1D(256, 3, padding='same', strides = 1, activation='relu')(embed)
    cnn = MaxPool1D(pool_size=4)(cnn)
    cnn = Flatten()(cnn)
    cnn = Dense(256)(cnn)
    cnn = Dropout(0.5)(cnn)
    
    rnn = Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1))(embed)
    rnn = Dense(256)(rnn)
    rnn = Dropout(0.5)(rnn)
    
    con = Concatenate(axis=-1)([cnn,rnn])

    main_output  = Dense(label_newnum, activation = 'softmax')(con)
    
    model = Model(inputs = main_input, outputs = main_output)
    #opt = Adadelta(lr = 0.05, rho = 0.95, epsilon = 1e-06)
    opt = 'adam'
    model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])
    model.summary()
    return model

def train():
    if pretrain == 0:
        model = build_model()
    else:
        model = load_model(model_name)
            
    num_instances = len(y_train)
    iter_per_epoch = int(num_instances / batch_size)
    iter_pct10 = int(iter_per_epoch / 10)
    if (num_instances % batch_size) > 0:
        iter_per_epoch = iter_per_epoch + 1
    batch_cutoff = [0]
    for i in range(iter_per_epoch - 1):
        batch_cutoff.append(batch_size * (i+1))
    batch_cutoff.append(num_instances)
    
    total_start_t = time.time()
    best_metrics = 0.0
    early_stop_counter = 0
    for e in range(num_epoch):
        rand_idxs = np.random.permutation(num_instances)
        print('\n#######')
        print('epoch' + str(e+1))
        print('#######')
        start_t = time.time()
        for i in range(iter_per_epoch):
            X_batch = []
            Y_batch = []
            
            for n in range(batch_cutoff[i],batch_cutoff[i+1]):
                X_batch.append(x_train[rand_idxs[n]])
                Y_batch.append(y_train[rand_idxs[n]])
            loss_and_metrics = model.train_on_batch(np.asarray(X_batch),np.asarray(Y_batch))
            if i % iter_pct10 == iter_pct10 - 1:
                print('Iteration ',i+1,' (',round(i*100/iter_per_epoch,2),'%) Train Loss: ',loss_and_metrics[0],'; Train accuracy: ',loss_and_metrics[1],'              ')
            else:
                print('Iteration ',i+1,' (',round(i*100/iter_per_epoch,2),'%) Train Loss: ',loss_and_metrics[0],'; Train accuracy: ',loss_and_metrics[1],'              ',end='\r')
                
        print('Train Loss: ',loss_and_metrics[0],'; Train accuracy: ',loss_and_metrics[1])
        loss_and_metrics = model.evaluate(x_test, y_test, batch_size)
        print('Test Loss: ',loss_and_metrics[0],': Test accuracy: ',loss_and_metrics[1])
        
        if loss_and_metrics[1] >= best_metrics:
            best_metrics = loss_and_metrics[1]
            print('Save best score!! '+str(loss_and_metrics[1]))
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            
        print('Elapsed time in epoch ' + str(e+1) + ': ' + str(time.time() - start_t))
        
        if (e+1) % save_every == 0:
            model_path = os.path.join(model_dir, 'model-%d.h5' %(e+1))
            model.save(model_path)
            model_weights_path = os.path.join(model_dir, 'model_weights-%d.h5' %(e+1))
            model.save_weights(model_weights_path)
            print('Saved model %s!' %str(e+1))
            
        if early_stop_counter >= patience:
            print('Stop by early stopping')
            print('Best score: ' + str(best_metrics))
            break
            
    print('Elapsed time in total: ' + str(time.time() - total_start_t))

train()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 1024)          0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1024, 300)     2400000     input_2[0][0]                    
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 1024, 256)     230656      embedding_2[0][0]                
____________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)   (None, 256, 256)      0           conv1d_2[0][0]                   
___________________________________________________________________________________________